In [148]:
# Import libraries
import pandas as pd
import numpy as np
import panel as pn
from panel.interact import interact
pn.extension()

import hvplot.pandas

In [149]:
# Create DataFrame from csv file - Tobias
df = pd.read_csv("dkb.csv", sep=";", encoding = 'ISO-8859-1', skiprows=6)

df = df.drop(["Wertstellung","Buchungstext","Gläubiger-ID","BLZ","Kontonummer", 
              "Verwendungszweck","Mandatsreferenz","Kundenreferenz","Unnamed: 11"],axis=1)

# Rename columns
df = df.rename(columns={'Auftraggeber / Begünstigter':'Auftraggeber'})
df = df.rename(columns={'Betrag (EUR)':'Betrag'})

# Insert additional columns
df['Kategorie'] = 'Sonstiges'
df['Monat'] = 'Nicht definiert'
df['Jahr'] = 'Nicht definiert'
df['Ein/Ausgang'] = 'Eingang'

# Format Ein/Ausgang
df['Ein/Ausgang'] = np.where(df['Betrag'].str[0] == '-','Ausgang', df['Ein/Ausgang'])

# Format Betrag
df['Betrag'] = df['Betrag'].str.replace(r'\.', '', regex=True)
df['Betrag'] = df['Betrag'].str.replace(r'\,', '.', regex=True)
df['Betrag'] = df['Betrag'].astype(float)

# Format Buchungstag
df['Buchungstag'] = df['Buchungstag'].astype(str)

# Get year and month
df['Jahr'] = df['Buchungstag'].str.slice(6,10)
df['Monat'] = df['Buchungstag'].str.slice(3,5) 
df['Monat'] = np.where(df['Monat'] == '01','Januar', df['Monat'])           
df['Monat'] = np.where(df['Monat'] == '02','Februar', df['Monat'])       
df['Monat'] = np.where(df['Monat'] == '03','März', df['Monat'])
df['Monat'] = np.where(df['Monat'] == '04','April', df['Monat'])
df['Monat'] = np.where(df['Monat'] == '05','Mai', df['Monat'])
df['Monat'] = np.where(df['Monat'] == '06','Juni', df['Monat']) 
df['Monat'] = np.where(df['Monat'] == '07','Juli', df['Monat'])
df['Monat'] = np.where(df['Monat'] == '08','August', df['Monat']) 
df['Monat'] = np.where(df['Monat'] == '09','September', df['Monat'])
df['Monat'] = np.where(df['Monat'] == '10','Oktober', df['Monat'])        
df['Monat'] = np.where(df['Monat'] == '11','November', df['Monat'])
df['Monat'] = np.where(df['Monat'] == '12','Dezember', df['Monat'])

In [150]:
# Create DataFrame from csv file - Tobias & Julia
df_TJ = pd.read_csv("dkb_TJ.csv", sep=";", encoding = 'ISO-8859-1', skiprows=6)

df_TJ = df_TJ.drop(["Wertstellung","Buchungstext","Gläubiger-ID","BLZ","Kontonummer", 
              "Verwendungszweck","Mandatsreferenz","Kundenreferenz","Unnamed: 11"],axis=1)
# Rename columns
df_TJ = df_TJ.rename(columns={'Auftraggeber / Begünstigter':'Auftraggeber'})
df_TJ = df_TJ.rename(columns={'Betrag (EUR)':'Betrag'})

# Insert additional columns
df_TJ['Kategorie'] = 'Sonstiges'
df_TJ['Monat'] = 'Nicht definiert'
df_TJ['Jahr'] = 'Nicht definiert'
df_TJ['Ein/Ausgang'] = 'Eingang'

# Format Ein/Ausgang
df_TJ['Ein/Ausgang'] = np.where(df_TJ['Betrag'].str[0] == '-','Ausgang', df_TJ['Ein/Ausgang'])

# Format Betrag
df_TJ['Betrag'] = df_TJ['Betrag'].str.replace(r'\.', '', regex=True)
df_TJ['Betrag'] = df_TJ['Betrag'].str.replace(r'\,', '.', regex=True)
df_TJ['Betrag'] = df_TJ['Betrag'].astype(float)

# Format Buchungstag
df_TJ['Buchungstag'] = df_TJ['Buchungstag'].astype(str)

# Get year and month
df_TJ['Jahr'] = df_TJ['Buchungstag'].str.slice(6,10)
df_TJ['Monat'] = df_TJ['Buchungstag'].str.slice(3,5) 
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '01','Januar', df_TJ['Monat'])           
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '02','Februar', df_TJ['Monat'])       
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '03','März', df_TJ['Monat'])
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '04','April', df_TJ['Monat'])
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '05','Mai', df_TJ['Monat'])
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '06','Juni', df_TJ['Monat']) 
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '07','Juli', df_TJ['Monat'])
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '08','August', df_TJ['Monat']) 
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '09','September', df_TJ['Monat'])
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '10','Oktober', df_TJ['Monat'])        
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '11','November', df_TJ['Monat'])
df_TJ['Monat'] = np.where(df_TJ['Monat'] == '12','Dezember', df_TJ['Monat'])

In [151]:
# Categorization Tobias
# Lebensmittel

# Mittagessen
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Kommod'),'Mittagessen', df['Kategorie'])

# Jause
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Manga|MANGOLD|SPAR DANKT'),'Jause', df['Kategorie'])

# Handy
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'A1 Telekom'),'Handy', df['Kategorie'])

# Aktivitäten
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Bergbahn'),'Aktivitäten', df['Kategorie'])

# Investments
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'FLATEX'),'Investments', df['Kategorie'])

# Sportartikel
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'SPORTH.'),'Sportartikel', df['Kategorie'])

# Krankenversicherung
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'GENERALI'),'Krankenversicherung', df['Kategorie'])

# Klamotten
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Facona|Garzon|HM|Tretter-Schuhe|Oberpollinger|HIRMER'),'Klamotten', df['Kategorie'])

# Gemeinschaftskonto
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'JULIA BEER'),'Gemeinschaftskonto', df['Kategorie'])

# Pantec
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Pantec'),'Pantec', df['Kategorie'])

In [152]:
# Categorization Tobias & Julia
# Lebensmittel
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'SPAR DANKT|foodspring'),'Lebensmittel', df_TJ['Kategorie'])

# Jause
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'Manga|MANGOLD|Jahnhalle|RUFFS|HELLDONES|Suppenkuche|Skihuette'),'Jause', df_TJ['Kategorie'])

# Aktivitäten
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'Bergbahn|Ski Arlberg'),'Aktivitäten', df_TJ['Kategorie'])

# Auto
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'JET|SHELL|PARKEN|PARKGARAGE'),'Auto', df_TJ['Kategorie'])

# Drogerie
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'DM-Fil'),'Drogerie', df_TJ['Kategorie'])

# Wohnungseinrichtung
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'Motel a Miio|VOSSEN|ROTHO-SHOP'),'Wohnungseinrichtung', df_TJ['Kategorie'])

# Julia
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'Julia Beer'),'Julia', df_TJ['Kategorie'])

# Tobias
df_TJ['Kategorie'] = np.where(df_TJ['Auftraggeber'].str.contains(
    'TOBIAS BURTSCHER'),'Tobias', df_TJ['Kategorie'])

In [ ]:
# Common widgets

ToggleInOut = pn.widgets.RadioButtonGroup(options=['Ausgaben','Einnahmen'], button_type='success')
Summe = pn.widgets.TextInput(name="Summe", value="0")
CheckSonstiges = pn.widgets.Checkbox(name='Details einblenden')

In [ ]:
# Account Tobias
YearT = pn.widgets.Select(name='Jahr auswählen', options=['2023'])
MonthT = pn.widgets.Select(name='Monat auswählen', options=['Alle','Januar','Februar','März','April','Mai','Juni','Juli','August','September','Oktober','November','Dezember'])
CategoryInT = pn.widgets.Select(name='Kategorie auswählen', options=['Alle','Pantec','Krankenversicherung'])
CategoryOutT = pn.widgets.Select(name='Kategorie auswählen', options=['Alle','Mittagessen', 'Jause', 'Handy', 'Aktivitäten', 'Sportartikel', 'Krankenversicherung', 'Klamotten', 'Gemeinschaftskonto'])

# Callback function
@pn.depends(ToggleInOut, YearT.param.value, MonthT.param.value, CategoryInT.param.value, CategoryOutT.param.value, CheckSonstiges)
def update_accountT(InOut, year, month, categoryInT, categoryOutT, sonstiges):

    # Income year total (T)
    if InOut == 'Einnahmen' and month == 'Alle' and categoryInT == 'Alle':
        data = df[(df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Eingang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Einnahmen " + year, height=400, grid=True)
        table = df[(df['Jahr'] == year) & (df['Kategorie'] == 'Sonstiges') & (df['Ein/Ausgang'] == 'Eingang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))
    
    # Expenses year total (T)
    elif InOut == 'Ausgaben' and month == 'Alle' and categoryOutT == 'Alle':
        data = df[(df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Ausgang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data[data['Kategorie'].str.contains("Investments") == False]
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Ausgaben " + year, height=400, grid=True)   
        table = df[(df['Jahr'] == year) & (df['Kategorie'] == 'Sonstiges') & (df['Ein/Ausgang'] == 'Ausgang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))

    # Income month total (T)
    elif InOut == 'Einnahmen' and month != 'Alle' and categoryInT == 'Alle':
        
        data = df[(df['Monat'] == month) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Eingang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Einnahmen " + month + " " + year, height=400, grid=True)
        table = df[(df['Monat'] == month) & (df['Jahr'] == year) & (df['Kategorie'] == 'Sonstiges') & (df['Ein/Ausgang'] == 'Eingang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))

    # Expenses month total (T)
    elif InOut == 'Ausgaben' and month != 'Alle' and categoryOutT == 'Alle':
        
        data = df[(df['Monat'] == month) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Ausgang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data[data['Kategorie'].str.contains("Investments") == False]
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Ausgaben " + month + " " + year, height=400, grid=True)
        table = df[(df['Monat'] == month) & (df['Jahr'] == year) & (df['Kategorie'] == 'Sonstiges') & (df['Ein/Ausgang'] == 'Ausgang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))

    # Income/Expenses category total (T)
    elif month == 'Alle' and (categoryInT != 'Alle' or categoryOutT != 'Alle') :
        
        if InOut == 'Einnahmen': 
            data = df[(df['Kategorie'] == categoryInT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Eingang')]
        elif InOut == 'Ausgaben': 
            data = df[(df['Kategorie'] == categoryOutT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Ausgang')]

        data = data.groupby('Monat')['Betrag'].sum().reset_index()
        data['Betrag'] = data['Betrag'].abs()
        
        if InOut == 'Einnahmen': 
            data = data.hvplot.bar(x='Monat', y='Betrag', xlabel='', ylabel='', title="Einnahmen " + categoryInT + " " + year, height=400, grid=True)       
            table = df[(df['Kategorie'] == categoryInT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Eingang')]
            table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Einnahmen " + categoryInT + " " + year, height=350)
            if sonstiges == True:
                plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))
            else:
                plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))
            
        elif InOut == 'Ausgaben': 
            data = data.hvplot.bar(x='Monat', y='Betrag', xlabel='', ylabel='', title="Ausgaben " + categoryOutT + " " + year, height=400, grid=True)
            table = df[(df['Kategorie'] == categoryOutT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Ausgang')]
            table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Ausgaben " + categoryOutT + " " + year, height=350)
            if sonstiges == True:
                plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))
            else:
                plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))

    # Income category month (T)
    elif InOut == 'Einnahmen' and month != 'Alle' and categoryInT != 'Alle':
        
        data = df[(df['Monat'] == month) & (df['Kategorie'] == categoryInT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Eingang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title= "Einnahmen " + categoryInT + " " + month +  " " + year, height=400, grid=True)
        table = df[(df['Monat'] == month) & (df['Kategorie'] == categoryInT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Eingang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title= "Einnahmen " + categoryInT + " " + month +  " " + year, height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryInT, Summe, CheckSonstiges))

    # Expenses category month (T)
    elif InOut == 'Ausgaben' and month != 'Alle' and categoryOutT != 'Alle':
        
        data = df[(df['Monat'] == month) & (df['Kategorie'] == categoryOutT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Ausgang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title= "Ausgaben " + categoryOutT + " " + month +  " " + year, height=400, grid=True)
        table = df[(df['Monat'] == month) & (df['Kategorie'] == categoryOutT) & (df['Jahr'] == year) & (df['Ein/Ausgang'] == 'Ausgang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title= "Ausgaben " + categoryOutT + " " + month +  " " + year, height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearT, MonthT, CategoryOutT, Summe, CheckSonstiges))

    my_sum = data['Betrag'].sum()
    my_sum = int(my_sum)
    Summe.value = str(my_sum) + " €"
    
    return plot

In [ ]:
# Account Tobias & Julia

YearTJ = pn.widgets.Select(name='Jahr auswählen', options=['2023'])
MonthTJ = pn.widgets.Select(name='Monat auswählen', options=['Alle','Januar','Februar','März','April','Mai','Juni','Juli','August','September','Oktober','November','Dezember'])
CategoryInTJ = pn.widgets.Select(name='Kategorie auswählen', options=['Alle','Julia','Tobias'])
CategoryOutTJ = pn.widgets.Select(name='Kategorie auswählen', options=['Alle','Lebensmittel','Jause','Aktivitäten','Drogerie','Auto','Wohnungseinrichtung', 'Miete', 'Stadtwerke', 'Internet und GIS'])

# Callback function
@pn.depends(ToggleInOut, YearTJ.param.value, MonthTJ.param.value, CategoryInTJ.param.value, CategoryOutTJ.param.value, CheckSonstiges)
def update_accountTJ(InOut, year, month, categoryInTJ, categoryOutTJ, sonstiges):

    # Income year total (TJ)
    if InOut == 'Einnahmen' and month == 'Alle' and categoryInTJ == 'Alle':
        data = df_TJ[(df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Eingang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Einnahmen " + year, height=400, grid=True)
        table = df_TJ[(df_TJ['Jahr'] == year) & (df_TJ['Kategorie'] == 'Sonstiges') & (df_TJ['Ein/Ausgang'] == 'Eingang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))
    
    # Expenses year total (TJ)
    elif InOut == 'Ausgaben' and month == 'Alle' and categoryOutTJ == 'Alle':
        data = df_TJ[(df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Ausgang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Ausgaben " + year, height=400, grid=True)   
        table = df_TJ[(df_TJ['Jahr'] == year) & (df_TJ['Kategorie'] == 'Sonstiges') & (df_TJ['Ein/Ausgang'] == 'Ausgang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryOutTJ, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryOutTJ, Summe, CheckSonstiges))

    # Income month total (TJ)
    elif InOut == 'Einnahmen' and month != 'Alle' and categoryInTJ == 'Alle':
        
        data = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Eingang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Einnahmen " + month + " " + year, height=400, grid=True)
        table = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Jahr'] == year) & (df_TJ['Kategorie'] == 'Sonstiges') & (df_TJ['Ein/Ausgang'] == 'Eingang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))

    # Expenses month total (TJ)
    elif InOut == 'Ausgaben' and month != 'Alle' and categoryOutTJ == 'Alle':
        
        data = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Ausgang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Ausgaben " + month + " " + year, height=400, grid=True)
        table = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Jahr'] == year) & (df_TJ['Kategorie'] == 'Sonstiges') & (df_TJ['Ein/Ausgang'] == 'Ausgang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Kategorie Sonstiges", height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryOutTJ, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryOutTJ, Summe, CheckSonstiges))

    # Income/Expenses category total (TJ)
    elif month == 'Alle' and (categoryInTJ != 'Alle' or categoryOutTJ != 'Alle') :
        
        if InOut == 'Einnahmen': 
            data = df_TJ[(df_TJ['Kategorie'] == categoryInTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Eingang')]
        elif InOut == 'Ausgaben': 
            data = df_TJ[(df_TJ['Kategorie'] == categoryOutTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Ausgang')]

        data = data.groupby('Monat')['Betrag'].sum().reset_index()
        data['Betrag'] = data['Betrag'].abs()
        
        if InOut == 'Einnahmen': 
            data = data.hvplot.bar(x='Monat', y='Betrag', xlabel='', ylabel='', title="Einnahmen " + categoryInTJ + " " + year, height=400, grid=True)       
            table = df_TJ[(df_TJ['Kategorie'] == categoryInTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Eingang')]
            table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Einnahmen " + categoryInTJ + " " + year, height=350)
            if sonstiges == True:
                plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))
            else:
                plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))
            
        elif InOut == 'Ausgaben': 
            data = data.hvplot.bar(x='Monat', y='Betrag', xlabel='', ylabel='', title="Ausgaben " + categoryOutTJ + " " + year, height=400, grid=True)
            table = df_TJ[(df_TJ['Kategorie'] == categoryOutTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Ausgang')]
            table = table.hvplot.table(['Auftraggeber', 'Betrag'], title="Ausgaben " + categoryOutTJ + " " + year, height=350)
            if sonstiges == True:
                plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryOutTJ, Summe, CheckSonstiges))
            else:
                plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryOutTJ, Summe, CheckSonstiges))

    # Income category month (TJ)
    elif InOut == 'Einnahmen' and month != 'Alle' and categoryInTJ != 'Alle':
        
        data = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Kategorie'] == categoryInTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Eingang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title= "Einnahmen " + categoryInTJ + " " + month +  " " + year, height=400, grid=True)
        table = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Kategorie'] == categoryInTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Eingang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title= "Einnahmen " + categoryInTJ + " " + month +  " " + year, height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))

    # Expenses category month (TJ)
    elif InOut == 'Ausgaben' and month != 'Alle' and categoryOutTJ != 'Alle':
        
        data = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Kategorie'] == categoryOutTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Ausgang')]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        data = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title= "Ausgaben " + categoryOutTJ + " " + month +  " " + year, height=400, grid=True)
        table = df_TJ[(df_TJ['Monat'] == month) & (df_TJ['Kategorie'] == categoryOutTJ) & (df_TJ['Jahr'] == year) & (df_TJ['Ein/Ausgang'] == 'Ausgang')]
        table = table.hvplot.table(['Auftraggeber', 'Betrag'], title= "Ausgaben " + categoryOutTJ + " " + month +  " " + year, height=350)
        if sonstiges == True:
            plot = pn.Row(data, table, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))
        else:
            plot = pn.Row(data, pn.Column(ToggleInOut, YearTJ, MonthTJ, CategoryInTJ, Summe, CheckSonstiges))

    my_sum = data['Betrag'].sum()
    my_sum = int(my_sum)
    Summe.value = str(my_sum) + " €"
    
    return plot

In [ ]:
# Select account with radio button

ToggleAccount = pn.widgets.RadioButtonGroup(options=['Gemeinschaftskonto','Konto Tobias'], button_type='success')

# Callback function
@pn.depends(ToggleAccount)
def setMainArea(account):
    if account == 'Gemeinschaftskonto':
        return update_accountTJ
    if account == 'Konto Tobias':
        return update_accountT

In [ ]:
# Show dashboard

dashboard = pn.template.FastListTemplate(
    title="Finanz Dashboard",
    sidebar=[
        pn.Column(ToggleAccount),
        pn.pane.PNG('money.png', sizing_mode='scale_width'),
        pn.pane.Markdown("## *Ein- und Ausgänge sind gesondert zu betrachten! <br><br> Julia Beer, 08.04.2023*")
    ],
    main=[
        pn.Row(setMainArea)
    ]
)

dashboard.show()